In [2]:
import sqlite3
import math
from collections import defaultdict
import os

In [3]:
path_tpl = R"C:\Users\lwnpk\OneDrive\Projects\HTML\WayneLin92.github.io\ss-fb42729d\AdamsSS_tmp\index_tpl.html"
path_html = R"C:\Users\lwnpk\OneDrive\Projects\HTML\WayneLin92.github.io\ss-fb42729d\AdamsSS_tmp\index.html"
path_js = R"C:\Users\lwnpk\OneDrive\Projects\HTML\WayneLin92.github.io\ss-fb42729d\AdamsSS_tmp\data.js"
class Config:
    bullets_tilt_angle = -17 / 180 * math.pi
    bullets_radius = 0.08
cosA = math.cos(Config.bullets_tilt_angle)
sinA = math.sin(Config.bullets_tilt_angle)


In [4]:
def str2array(str_array: str):
    return [int(i) for i in str_array.split(",")] if len(str_array) > 0 else []

def get_complex_name(path):
    path = os.path.basename(path)
    names = ["S0", "C2", "Ceta", "Cnu", "Csigma"]
    for name in names:
        if path.startswith(name):
            return name
    raise ValueError(f"{path=} is not recognized")

In [5]:
def load_pi_basis(c, table, is_ring: bool):
    result = {"basis": [], "bullets": defaultdict(list), "bullets_ind": set(), "bullets_color": defaultdict(list)}
    index = 0
    sql = f"SELECT mon, s, t FROM {table} ORDER BY id"
    for str_mon, s, t in c.execute(sql):
        if is_ring:
            mon = list(map(int, str_mon.split(","))) if len(str_mon) > 0 else []
            mon1 = [[mon[i] // 2, mon[i + 1]] for i in range(0, len(mon) - 2, 2)]
            mon1.sort()
            mon = sum(mon1, [])
        else:
            mon = list(map(int, str_mon.split(",")))
            mon1 = [[mon[i] // 2, mon[i + 1]] for i in range(0, len(mon) - 3, 2)]
            mon1.sort()
            mon = sum(mon1, []) + [mon[-1], 1]

        result["basis"].append(mon)
        result["bullets"][(t - s, s)].append(index)
        if len(mon) == 2 and mon[1] == 1:
            color = "blue"
            result["bullets_ind"].add(index)
        else:
            color = "black"
        result["bullets_color"][(t - s, s)].append(color)
        index += 1
    return result

def load_pi_gen_names(c, table, letter):
    sql = f"SELECT id, name, t - s FROM {table} order by id"
    gens_stem = defaultdict(int)
    result = []
    for id, name, stem in c.execute(sql):
        if name:
            result.append(name)
        elif gens_stem[stem] == 0:
            if stem < 10:
                result.append(f"{letter}_{stem}")
            else:
                result.append(f"{letter}_{{{stem}}}")
        else:
            result.append(f"{letter}_{{{stem},{gens_stem[stem]}}}")
        gens_stem[stem] += 1
    return result

def load_pi_multiplications(c, table):
    sql = f"SELECT id1, id2, prod, O FROM {table}"
    result = []
    for id1, id2, prod, O in c.execute(sql):
        result.append((id1, id2, str2array(prod), O))
    return result

def load_ring(path_ring):
    is_ring = True
    conn_ring = sqlite3.connect(path_ring)
    c_ring = conn_ring.cursor()

    path_plot = path_ring.replace(".db", "_plot.db")
    conn_plot = sqlite3.connect(path_plot)
    c_plot = conn_plot.cursor()

    complex = "S0"

    result = load_pi_basis(c_ring, complex + "_pi_basis", is_ring)
    result["ring_gen_names"] = load_pi_gen_names(c_ring, complex + "_pi_generators", "\\mu")
    result["products"] = load_pi_multiplications(c_plot, complex + "_pi_basis_products")

    c_ring.close()
    conn_ring.close()
    c_plot.close()
    conn_plot.close()

    return result

def load_mod(path, path_ring):
    is_ring = False
    conn = sqlite3.connect(path)
    c = conn.cursor()

    conn_ring = sqlite3.connect(path_ring)
    c_ring = conn_ring.cursor()

    path_plot = path.replace(".db", "_plot.db")
    conn_plot = sqlite3.connect(path_plot)
    c_plot = conn_plot.cursor()

    complex = get_complex_name(path)

    result = load_pi_basis(c, complex + "_pi_basis", is_ring)
    result["ring_gen_names"] = load_pi_gen_names(c_ring, "S0_pi_generators", "\\mu")
    result["mod_gen_names"] = load_pi_gen_names(c, complex + "_pi_generators", "\\iota")
    result["products"] = load_pi_multiplications(c_plot, complex + "_pi_basis_products")

    c.close()
    conn.close()
    c_ring.close()
    conn_ring.close()
    c_plot.close()
    conn_plot.close()

    return result


In [6]:
def smoothen(radius):
    """Make the distribution of radius smooth """
    radius_ub = defaultdict(lambda: 0.1)  # upper bound of radius
    b_changed = True
    while b_changed:
        for x, y in radius:
            r = radius[(x, y)] * 1.085
            r1 = radius[(x, y)] * 1.13
            radius_ub[(x + 1, y)] = min(radius_ub[(x + 1, y)], r)
            radius_ub[(x - 1, y)] = min(radius_ub[(x - 1, y)], r)
            radius_ub[(x, y + 1)] = min(radius_ub[(x, y + 1)], r)
            radius_ub[(x, y - 1)] = min(radius_ub[(x, y - 1)], r)
            radius_ub[(x + 1, y + 1)] = min(radius_ub[(x + 1, y + 1)], r1)
            radius_ub[(x + 1, y - 1)] = min(radius_ub[(x + 1, y - 1)], r1)
            radius_ub[(x - 1, y + 1)] = min(radius_ub[(x - 1, y + 1)], r1)
            radius_ub[(x - 1, y - 1)] = min(radius_ub[(x - 1, y - 1)], r1)
        
        b_changed = False
        for x, y in radius:
            if radius[(x, y)] > radius_ub[(x, y)] * 1.005:
                radius[(x, y)] = radius_ub[(x, y)]
                b_changed = True

def get_radius(data):
    """Set the radius of bullets in each coordinate (x, y) """
    radius = defaultdict(lambda: Config.bullets_radius)
    for x, y in data["bullets"]:
        len_bullets_xy = len(data["bullets"][(x, y)])
        length_world = (len_bullets_xy - 1) * Config.bullets_radius * 3
        if (length_world > 1 - Config.bullets_radius * 6):
            length_world = 1 - Config.bullets_radius * 6
        sep_world = length_world / (len_bullets_xy - 1) if len_bullets_xy > 1 else Config.bullets_radius * 3
        r = sep_world / 3

        if r < radius[(x, y)]:
            radius[(x, y)] = r
    smoothen(radius)
    return radius

def export_bullets(data, radius):
    # Bullets. Compute `index2xyr` for lines
    tpl_bullets = ""
    index2xyrp = {}
    for x, y in data["bullets"]:
        r = radius[(x, y)]
        len_bullets_xy = len(data["bullets"][(x, y)])
        sep_world = r * 3
        length_world = (len_bullets_xy - 1) * sep_world

        bottom_right_x = x + length_world / 2 * cosA
        bottom_right_y = y + length_world / 2 * sinA
        for i, index in enumerate(data["bullets"][(x, y)]):
            cx = bottom_right_x - i * sep_world * cosA
            cy = bottom_right_y - i * sep_world * sinA
            str_fill = f'fill="{data["bullets_color"][(x, y)][i]}" ' if data["bullets_color"][(x, y)][i] != "black" else ''
            page = 200
            level = 9800
            str_class = 'b'
            tpl_bullets += f'<circle id=b{index} class={str_class} cx={cx:.6g} cy={cy:.6g} r={r:.6g} {str_fill}'\
                f'data-b={index} data-l={level} data-d="{None}" data-i={0} data-j={0} data-g={int(len(str_fill) > 0)} data-page={page}>'\
                    f'<title>({x:.6g}, {y:.6g}) id: {index}</title></circle>\n'
            index2xyrp[index] = (cx, cy, r, page)
    return tpl_bullets, index2xyrp

def export_lines(data, index2xyrp, lines, dashed_lines):
    # Plot the struct lines
    tpl_lines = ["", "", ""]
    extend_colors = ["red", "blue", "green"]
    for i in range(3):
        for i1, i2 in lines[i]:
            x1, y1, r1, p1 = index2xyrp[i1]
            x2, y2, r2, p2 = index2xyrp[i2]
            tpl_lines[i] += f'<line class=strt_l x1={x1:.6g} y1={y1:.6g} x2={x2:.6g} y2={y2:.6g} stroke-width={min(r1, r2) / 4:.6g} data-page={min(p1, p2)} />'
    for i in range(3):
        for i1, O in dashed_lines[i]:
            x1, y1, r1, p1 = index2xyrp[i1]
            x2, y2, r2, p2 = round(x1) + 2 ** i - 1, O, r1, p1
            to_draw = False
            if (x2, y2) in data["bullets"]:
                i2 = data["bullets"][(x2, y2)][0]
                x2, y2, r2, p2 = index2xyrp[i2]
                to_draw = True
            else:
                if i1 in data["bullets_ind"]:
                    to_draw = True
            if to_draw:
                if i == 0:
                    tpl_lines[i] += f'<path class=dashed_l d="M {x1:.6g} {y1:.6g} C {x1 + 0.3:.6g} {y1 * 0.7 + y2 * 0.3:.6g}, {x2 + 0.3:.6g} {y1 * 0.3 + y2 * 0.7:.6g}, {x2:.6g} {y2:.6g}" '\
                    f'stroke-width={min(r1, r2) / 4:.6g} stroke-dasharray="0.2,0.2" fill="transparent" stroke="{extend_colors[i]}" />\n'
                else:
                    tpl_lines[i] += f'<path class=dashed_l d="M {x1:.6g} {y1:.6g} C {x1 + 0.5:.6g} {y1 + 0.5 / (2 ** i - 1):.6g}, {x2 - (2 ** i - 1) / 4 / (y2 - y1):.6g} {y2 - 0.5:.6g}, {x2:.6g} {y2:.6g}" '\
                    f'stroke-width={min(r1, r2) / 4:.6g} stroke-dasharray="0.2,0.2" fill="transparent" stroke="{extend_colors[i]}" />\n'
    return tpl_lines
    

def export_gen_names(data, is_ring: bool):
    content_js = "const gen_names = [\n"
    for name in data["ring_gen_names"]:
        name1 = name.replace('\\', '\\\\')
        content_js += f' "{name1}",\n'
    if not is_ring:
        for name in data["mod_gen_names"]:
            name1 = name.replace('\\', '\\\\')
            content_js += f' "{name1}",\n'
    content_js += "];\n\n"
    return content_js

def export_basis(data, is_ring: bool):
    content_js = "const basis = [\n"
    for mon in data["basis"]:
        if not is_ring:
            mon[-2] += len(data["ring_gen_names"])
        content_js += f" {mon},\n"
    content_js += f"];\n\n"
    return content_js

def export_basis_prod(data):
    content_js = "const basis_prod = {\n"
    lines = [[], [], []]  # h0, h1, h2
    dashed_lines = [[], [], []]  # h0, h1, h2
    b2g = {1: 0, 3: 1, 7: 2}  # basis_id to gen_id
    for id1, id2, prod, O in data["products"]:
        if len(prod) > 0:
            content_js += f' "{id1},{id2}": {prod},\n'
            if id1 in b2g:
                for id3 in prod:
                    lines[b2g[id1]].append((id2, id3))
        if id1 in b2g and O != -1:
            dashed_lines[b2g[id1]].append((id2, O))
    content_js += "};\n\n"
    return content_js, lines, dashed_lines

def export_pi(data, is_ring: bool):
    tpl_title = "Adams Spectral Sequence"
    
    # js
    content_js = ""
    content_js += export_gen_names(data, is_ring)
    content_js += export_basis(data, is_ring)
    content_js1, lines, dashed_lines = export_basis_prod(data)
    content_js += content_js1
    
    # html
    radius = get_radius(data)
    tpl_bullets, index2xyrp = export_bullets(data, radius)
    tpl_lines = export_lines(data, index2xyrp, lines, dashed_lines)
    
    with open(path_tpl, encoding="utf-8") as fp:
        content_tpl = fp.read()

    content = content_tpl.replace("title:13dd3d15", tpl_title)
    content = content.replace("bullets:b8999a4e", tpl_bullets)

    content = content.replace("h0_lines:839e707e", tpl_lines[0])
    content = content.replace("h1_lines:2b707f82", tpl_lines[1])
    content = content.replace("h2_lines:e766a4f6", tpl_lines[2])

    content = content.replace("d2_lines:d9dec788", "")
    content = content.replace("d3_lines:1a07bf89", "")
    content = content.replace("d4_lines:7d935e97", "")
    content = content.replace("d5_lines:f6d7f992", "")
    content = content.replace("d6_lines:05c49c4a", "")

    with open(path_js, "w") as file:
        file.write(content_js)
    with open(path_html, "w", encoding="utf-8") as file:
        file.write(content)


In [9]:
path = R'C:\Users\lwnpk\Documents\Projects\algtop_cpp_build\bin\Release\benchmark\C2_AdamsSS_t100.db'
path_ring = R'C:\Users\lwnpk\Documents\Projects\algtop_cpp_build\bin\Release\benchmark\S0_AdamsSS_t100.db'

In [10]:
data = load_ring(path_ring)
export_pi(data, True)

In [11]:
data = load_mod(path, path_ring)
export_pi(data, False)